In [2]:
import pandas as pd
import gensim
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()
import numpy as np
from pprint import pprint
import string
import re
%matplotlib inline
np.set_printoptions(suppress=True)

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [19]:
import keras

In [3]:
df = pd.read_csv("data/train_binary.csv")

In [33]:
df["y"] = 0
df.loc[df["Category"] == "cancer", "y"] = 1

In [34]:
df.head()

,Id,Title,Abstract,Category,y
0,746,SHP2 promotes laryngeal cancer growth through ...,The overall survival rate and prognosis of pat...,cancer,1
1,421,Response to Crizotinib Observed in Lung Adenoc...,MMNG HOS Transforming gene (MET) is an importa...,cancer,1
2,261,Genome-wide profiling of genetic synthetic let...,"Small-molecule inhibitors of PARP1/2, such as ...",cancer,1
3,644,MMP2 and MMP9 serum levels are associated with...,Addition of bevacizumab to trastuzumab-based n...,cancer,1
4,1495,Src Family Kinases Modulate the Loss of Endoth...,Activation of Src Family Kinase (SFK) signalin...,nonCancer,0


In [6]:
documents = df["Title"]


In [7]:
stoplist = list(gensim.parsing.preprocessing.STOPWORDS)
def word_is_valid(word):
    return word not in stoplist and len(word) > 2 and len(word) < 20

In [8]:
texts = [[word for word in document.lower().split() if word_is_valid(word)] for document in documents]

In [57]:
dictionary = gensim.corpora.Dictionary(texts)
print(dictionary)

Dictionary(7963 unique tokens: ['angioimmunoblastic', 'glioblastoma', 'dermacell', 'nanogap', 'hcv']...)


In [58]:
dictionary.filter_extremes(no_above=0.9, no_below=5)
print(dictionary)

Dictionary(755 unique tokens: ['glioblastoma', 'retinoic', 'combination', 'family', 'activating']...)


In [21]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
count_vect = CountVectorizer(vocabulary=dictionary.token2id)
X_train_counts = count_vect.fit_transform(documents)
X_train_counts.shape

(2155, 755)

In [49]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [50]:
X_train_tfidf.shape

(2155, 755)

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
lr = LogisticRegression()

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, df["y"])

In [53]:
X_train

<1616x755 sparse matrix of type '<class 'numpy.float64'>'
	with 10589 stored elements in Compressed Sparse Row format>

In [54]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
lr.score(X_test, y_test)

0.92393320964749537